In [2]:
import numpy as np
import keras
import tensorflow as tf

#匯入模型
model = keras.models.load_model("./0903_size32.h5")
model.trainable = False
model.compile(
    # Technical note: when using embedding layers, I highly recommend using one of the optimizers
    # found  in tf.train: https://www.tensorflow.org/api_guides/python/train#Optimizers
    # Passing in a string like 'adam' or 'SGD' will load one of keras's optimizers (found under 
    # tf.keras.optimizers). They seem to be much slower on problems like this, because they
    # don't efficiently handle sparse gradient updates.
    tf.train.AdamOptimizer(0.005),
    loss='MSE',
    metrics=['MAE'],
)
#model.summary()
temp = model.predict([np.array([3]).reshape(-1,1), np.array([3]).reshape(-1,1)])
del temp


embedd_movies = model.get_layer(name = "Embedd_movies")

(w,) = embedd_movies.get_weights()
movie_embedding_size = w.shape[1]

from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

#匯入資料
import pandas as pd
movies = pd.read_csv("/home/fio/Python/movielens_20m/Data/movie_pro.csv")
movies = movies[movies.n_rating>1000]
moviename = movies.title

movies = movies[movies.n_rating>1000]

movie_id_list = np.array(movies.movieId)
movie_id_len = len(movie_id_list)
title_list = movies['title'].copy()
title_list_total = pd.DataFrame(data = title_list)


#get similar
kv = WordEmbeddingsKeyedVectors(movie_embedding_size)
kv.add(
    movies['title'].values,
    w[movies.movieId]
)

#kv.most_similar(name)



#定義API

from flask import Flask, request, abort
app = Flask(__name__)

from waitress import serve

@app.route("/callback")
def callback():
    return 'OK'

@app.route("/api/gs/<string:name>")
def similar_first(name):
    try: name = np.array(moviename[moviename.str.contains(name.title())])[0]
    except IndexError:return "Movie not found."
    #print("Most similar with %s:"%(name))
    string = 'According to the rates by audience.<br>The movie %s is more similar to:<br><br>'%(name)
    try:
        for i,j in kv.most_similar(name):
            string+="%s, %f <br/>"%(i,j)
    except KeyError: 
        print(" Error Key : %s"%(name))
        return "Key Error, Try another word."
    return string


@app.route("/api/rm/<int:userId>")
def recommand(userId):
    num = 5
    ids = [userId]*movie_id_len
    result = model.predict([ids, movie_id_list])
    title_list = title_list_total.copy()
    title_list['predict'] = result+3.7 
    title_list = title_list.sort_values("predict", ascending=False).head(num)
    rating_list = list(title_list.predict)
    title_list = list(title_list.title)
    string = ("User %d will most likely to like:<br/>"%(ids[0]))
    for i in range(len(title_list)):
        string+=(" %.1f    %s <br>"%(rating_list[i], title_list[i]))
    return string

@app.route("/api/sf/<string:inputs>")
def predict_s(inputs):
    user = inputs.split("<>")[0]
    movie_name = inputs.split("<>")[1:]
    print(inputs)
    user = [user] * len(movie_name)
    movie_ids = []
    movie_name_list = []
    for i in range(len(movie_name)):
        try: 
            temp_df = movies[movies.title.str.contains(movie_name[i].title())].head(1)
        except IndexError:
            return "Movie not found."
        try: movie_name_list.append(str(temp_df.title)[3:].replace("    ","").replace("\nName: title, dtype: object",""))
        except TypeError:
            return "Movie not found."
        try: movie_ids.append(int(temp_df.movieId))
        except TypeError:
            return "Movie not found."
    #print(movie_name)
    result = model.predict([user, movie_ids]) + 3.7
    string = "User %d to below movies rating prediction is:<br>"%(int(user[0]))
    for i in range(len(movie_name_list)):
        string += "%.2f - %s<br>"%(result[i], movie_name_list[i])
    return string
    
    
#啟動server
import sys 
print(sys.version)
if __name__ == "__main__":
    serve(app, host = '0.0.0.0', port = 12347)

/home/fio/anaconda3/envs/cpu/lib/python3.7/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]
Serving on http://0.0.0.0:12350


http://114.32.20.219:12347/api/gs/Thor

http://114.32.20.219:12347/api/rm/1

http://114.32.20.219:12347/api/sf/1%3C%3EIron%20Man%3C%3EToy%20Story